In [1]:
# Install necessary libraries
!pip install transformers


Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install --upgrade urllib3[secure] cryptography


Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install urllib3==1.25.10


Defaulting to user installation because normal site-packages is not writeable
  Using cached urllib3-1.25.10-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.0
    Uninstalling urllib3-2.2.0:
      Successfully uninstalled urllib3-2.2.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
google-auth 2.23.1 requires urllib3>=2.0.5, but you have urllib3 1.25.10 which is incompatible.
selenium 4.12.0 requires urllib3[socks]<3,>=1.26, but you have urllib3 1.25.10 which is incompatible.
anaconda-client 1.11.3 requires urllib3>=1.26.4, but you have urllib3 1.25.10 which is incompatible.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.31.0 which is incompatible.


In [4]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [1]:
# Import libraries
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from transformers import BertTokenizer, BertModel
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from nltk.stem import PorterStemmer, WordNetLemmatizer


In [2]:
cancer_data=pd.read_csv('lung_cancer_dataset.csv')
cancer_data.head()

,Patient ID,Age,Gender,Smoking History,Family History,Diagnosis Date,Cancer Stage,Histology Type,Tumor Size,Metastasis,...,Symptom 1,Symptom 2,Symptom 3,Response to Treatment,DFS (months),OS (months),Adverse Events,HRQoL Assessment,Follow-up Appointments,Patient-reported Outcomes
0,P001,70,Male,Smoker,No,20/04/2019,Stage II,Adenocarcinoma,3.93,Brain,...,Loss of appetite,Headaches,Fatigue,Complete Response,60,34,"Diarrhea, Neutropenia, Fatigue, Vomiting, Muco...",Improved,"['2024-04-07', '2025-12-29', '2025-07-05', '20...",Maintaining a positive outlook despite challen...
1,P002,36,Female,Ex-Smoker,No,07/08/2020,Stage I,Small Cell,7.68,Lung,...,Loss of appetite,Persistent infections,Shortness of breath,Partial Response,18,48,"Mucositis, Diarrhea, Fatigue, Hypothyroidism",Stable,"['2028-06-03', '2028-06-07', '2028-04-04', '20...",Having challenges with daily activities due to...
2,P003,72,Female,Non-Smoker,Yes,29/12/2022,Stage II,Small Cell,6.26,Brain,...,Difficulty swallowing,Unexplained weight loss,"Neurological symptoms (e.g., dizziness, weakness)",Progression,53,53,"Neutropenia, Fatigue",Improved,"['2024-02-18', '2026-11-08', '2024-09-07', '20...",Feeling generally unwell and fatigued.
3,P004,77,Female,Non-Smoker,No,21/05/2023,Stage III,Adenocarcinoma,5.39,Brain,...,Headaches,Joint pain,Fatigue,Complete Response,55,70,Pneumonitis,Stable,"['2028-03-04', '2025-12-16', '2024-12-16', '20...",Experiencing side effects from treatment.
4,P005,37,Female,Ex-Smoker,Yes,01/08/2020,Stage IV,Adenocarcinoma,2.51,Liver,...,Difficulty swallowing,"Neurological symptoms (e.g., dizziness, weakness)",Persistent cough,Complete Response,35,43,"Nausea, Anemia, Vomiting",Improved,"['2024-05-11', '2025-10-30', '2024-03-03', '20...",Struggling with the impact of the diagnosis on...


In [7]:
!pip install urllib3

Defaulting to user installation because normal site-packages is not writeable


In [4]:
cancer_data.isna().sum()

Patient ID                       0
Age                              0
Gender                           0
Smoking History                  0
Family History                   0
Diagnosis Date                   0
Cancer Stage                     0
Histology Type                   0
Tumor Size                       0
Metastasis                       0
Treatment Type                   0
Geneset                          0
Genetic Mutations                0
Blood Markers                    0
IHC Results                      0
Radiation Therapy Details        0
Chemotherapy Regimen             0
Clinical Trial Participation     0
Comorbidities                   29
ECOG Performance Status          0
Symptom 1                        0
Symptom 2                        0
Symptom 3                        0
Response to Treatment            0
DFS (months)                     0
OS (months)                      0
Adverse Events                  14
HRQoL Assessment                 0
Follow-up Appointmen

In [9]:
cancer_data.shape

(200, 30)

In [10]:
cancer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 30 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Patient ID                    200 non-null    object 
 1   Age                           200 non-null    int64  
 2   Gender                        200 non-null    object 
 3   Smoking History               200 non-null    object 
 4   Family History                200 non-null    object 
 5   Diagnosis Date                200 non-null    object 
 6   Cancer Stage                  200 non-null    object 
 7   Histology Type                200 non-null    object 
 8   Tumor Size                    200 non-null    float64
 9   Metastasis                    200 non-null    object 
 10  Treatment Type                200 non-null    object 
 11  Geneset                       200 non-null    object 
 12  Genetic Mutations             200 non-null    object 
 13  Blood

In [11]:
cancer_data=cancer_data.dropna()

In [12]:
cancer_data.shape

(158, 30)

In [13]:
!pip install --upgrade transformers


Defaulting to user installation because normal site-packages is not writeable


In [14]:
# Sample Medical Transcript
user_inputs = []
while True:
    user_input = input("Enter a user input (or 'quit' to stop): ")
    if user_input.lower() == "quit":
        break
    user_inputs.append(user_input)
for i in user_inputs:
    # Tokenize using NLTK
    tokens = word_tokenize(i)
    tokenizer = word_tokenize
# Output the tokens
print(tokens)


Enter a user input (or 'quit' to stop):  XYZ is a 54-year-old African American male, who presents to the ED with a complaint of a two-week history of hoarseness that is not getting better. When questioned further, he admits that his usual morning "smoker's cough" is getting worse and that he has "given serious thought lately to quitting smoking in the near  future."  PMH:  Hypothyroidism 27 years  • Bilateral osteoarthritis of knees 8 years  • Depression 2 years  Iron deficiency anemia of unknown cause, 8 months  Seasonal allergic rhinitis since adolescence  Several episodes of mildly bloody sputum in past year that resolved in a few days; patient  did not seek medical help  No prior history of trauma or surgeries  • Had influenza vaccine last year  Last tetanus booster, 6 years ago.  Family history is negative for lung cancer; however, there is a positive family history for other types of cancer-paternal uncle with colorectal cancer, one niece with non-Hodgkin's lymphoma, and one niec

In [15]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
lemmatizer=WordNetLemmatizer()

In [16]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [17]:
from nltk.corpus import stopwords
import string
stops=set(stopwords.words('english'))
punctuations=list(string.punctuation)
stops.update(punctuations)
stops,string.punctuation

({'!',
  '"',
  '#',
  '$',
  '%',
  '&',
  "'",
  '(',
  ')',
  '*',
  '+',
  ',',
  '-',
  '.',
  '/',
  ':',
  ';',
  '<',
  '=',
  '>',
  '?',
  '@',
  '[',
  '\\',
  ']',
  '^',
  '_',
  '`',
  'a',
  'about',
  'above',
  'after',
  'again',
  'against',
  'ain',
  'all',
  'am',
  'an',
  'and',
  'any',
  'are',
  'aren',
  "aren't",
  'as',
  'at',
  'be',
  'because',
  'been',
  'before',
  'being',
  'below',
  'between',
  'both',
  'but',
  'by',
  'can',
  'couldn',
  "couldn't",
  'd',
  'did',
  'didn',
  "didn't",
  'do',
  'does',
  'doesn',
  "doesn't",
  'doing',
  'don',
  "don't",
  'down',
  'during',
  'each',
  'few',
  'for',
  'from',
  'further',
  'had',
  'hadn',
  "hadn't",
  'has',
  'hasn',
  "hasn't",
  'have',
  'haven',
  "haven't",
  'having',
  'he',
  'her',
  'here',
  'hers',
  'herself',
  'him',
  'himself',
  'his',
  'how',
  'i',
  'if',
  'in',
  'into',
  'is',
  'isn',
  "isn't",
  'it',
  "it's",
  'its',
  'itself',
  'just',
  'll',


In [18]:
# Choose stemming or lemmatization based on your preference
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def normalize_text(text, stem=True):
    """
    Normalizes text by tokenizing, removing stop words, and applying stemming/lemmatization.
    """
    tokens = tokenizer(text.lower())
    filtered_tokens = [token for token in tokens if token not in stops]
    if stem:
        normalized_tokens = [stemmer.stem(token) for token in filtered_tokens]
    else:
        normalized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    return " ".join(normalized_tokens)


In [19]:
def clean_reviews(words):
    output_reviews=[]
    for w in words:
        if w.lower() not in stops:
            pos=pos_tag([w])
            #lemmatize the word
            clean_review=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_reviews.append(clean_review.lower())
    return output_reviews

In [20]:
medical_transcript=clean_reviews(tokens)
medical_transcript

['xyz',
 '54-year-old',
 'african',
 'american',
 'male',
 'present',
 'ed',
 'complaint',
 'two-week',
 'history',
 'hoarseness',
 'get',
 'well',
 'question',
 'admits',
 'usual',
 'morning',
 '``',
 'smoker',
 "'s",
 'cough',
 "''",
 'get',
 'bad',
 '``',
 'give',
 'serious',
 'thought',
 'lately',
 'quit',
 'smoking',
 'near',
 'future',
 "''",
 'pmh',
 'hypothyroidism',
 '27',
 'year',
 '•',
 'bilateral',
 'osteoarthritis',
 'knee',
 '8',
 'year',
 '•',
 'depression',
 '2',
 'year',
 'iron',
 'deficiency',
 'anemia',
 'unknown',
 'cause',
 '8',
 'month',
 'seasonal',
 'allergic',
 'rhinitis',
 'since',
 'adolescence',
 'several',
 'episode',
 'mildly',
 'bloody',
 'sputum',
 'past',
 'year',
 'resolve',
 'day',
 'patient',
 'seek',
 'medical',
 'help',
 'prior',
 'history',
 'trauma',
 'surgery',
 '•',
 'influenza',
 'vaccine',
 'last',
 'year',
 'last',
 'tetanus',
 'booster',
 '6',
 'year',
 'ago',
 'family',
 'history',
 'negative',
 'lung',
 'cancer',
 'however',
 'positive',


In [21]:
cancer_data["Symptom1"]=cancer_data["Symptom 1"]+" "+cancer_data["Symptom 2"]+" "+cancer_data["Symptom 3"]
cancer_data["Symptom1"][2]

'Difficulty swallowing Unexplained weight loss Neurological symptoms (e.g., dizziness, weakness)'

In [22]:
# Combine relevant text columns into a single 'symptoms' column
numeric_features = ['Age', 'Tumor Size',"ECOG Performance Status"]
textual_feature = 'medical_transcript'
text_columns = ['Geneset', 'Genetic Mutations', 'Gender', 'Smoking History',"Symptom 1","Symptom 2","Symptom 3",'Cancer Stage', 'Histology Type','Metastasis','Treatment Type',"Comorbidities","Clinical Trial Participation","IHC Results","Blood Markers","Family History"]
cancer_data['symptoms'] = cancer_data[text_columns].apply(lambda x: ' '.join(x), axis=1)

In [23]:
cancer_data['symptoms']

0      KRAS, HER2, PIK3CA KRAS G12V, HER2 HER2 Amplif...
2      CDKN2A, ATM, BCL2 CDKN2A CDKN2A Mutation, ATM ...
3      RET, KRAS, EGFR RET RET Fusion, KRAS G12V, EGF...
5      FGFR2, DDR2, SOX2 FGFR2 FGFR2 Amplification, D...
6      ERBB2, BRAF, PIK3CA ERBB2 HER2 Amplification, ...
                             ...                        
195    DDR2, FGFR2, NOTCH1 DDR2 DDR2 Mutation, FGFR2 ...
196    CDKN2A, NOTCH1, FGFR1 CDKN2A CDKN2A Mutation, ...
197    RET, KRAS, MET RET RET Fusion, KRAS G12V, MET ...
198    BRAF, ERBB2, PIK3CA BRAF D594G, ERBB2 HER2 Amp...
199    DDR2, FGFR1, NOTCH1 DDR2 DDR2 Mutation, FGFR1 ...
Name: symptoms, Length: 158, dtype: object

In [24]:
cancer_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 199
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Patient ID                    158 non-null    object 
 1   Age                           158 non-null    int64  
 2   Gender                        158 non-null    object 
 3   Smoking History               158 non-null    object 
 4   Family History                158 non-null    object 
 5   Diagnosis Date                158 non-null    object 
 6   Cancer Stage                  158 non-null    object 
 7   Histology Type                158 non-null    object 
 8   Tumor Size                    158 non-null    float64
 9   Metastasis                    158 non-null    object 
 10  Treatment Type                158 non-null    object 
 11  Geneset                       158 non-null    object 
 12  Genetic Mutations             158 non-null    object 
 13  Blood

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
# Assuming your target variables include 'Geneset', 'Genetic Mutations', 'Cancer Stage', 'Histology Type', and 'Metastasis'
target_columns = ['Geneset', 'Genetic Mutations', 'Cancer Stage', 'Histology Type', 'Metastasis']

# Features (excluding the target columns)
# Assuming your target variable is 'lung_cancer_status'
# Features (excluding the target columns)
X = cancer_data.drop(target_columns + ['Patient ID', 'Diagnosis Date', 'Chemotherapy Regimen', 'HRQoL Assessment', 
                                       'Symptom1', 'symptoms', 'Follow-up Appointments', 'Adverse Events', 
                                       'Radiation Therapy Details', 'Response to Treatment', 'DFS (months)', 'OS (months)'], 
                    axis=1) # Features
y = cancer_data[target_columns]  # Target variable

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Data Preprocessing for Gene and Transcript Data
# (Assuming you have numerical features in the gene and transcript data)
numeric_features = ['Age', 'Tumor Size', "ECOG Performance Status"]

# Standardize numerical features using StandardScaler
scaler = StandardScaler()
X_train[numeric_features] = scaler.fit_transform(X_train[numeric_features])
X_test[numeric_features] = scaler.transform(X_test[numeric_features])

# Data Preprocessing for Medical Transcript Data
# (Assuming you have categorical features in the medical transcript data)
text_columns = ['Gender', 'Smoking History', "Symptom 1", "Symptom 2", "Symptom 3",
                'Treatment Type', "Comorbidities", "Clinical Trial Participation", 
                "IHC Results", "Blood Markers", "Family History","Patient-reported Outcomes"]


# Encode categorical features using LabelEncoder
label_encoder = LabelEncoder()

for feature in text_columns:
    # Fit and transform the training set
    X_train[feature] = label_encoder.fit_transform(X_train[feature])

    # Handle unknown labels in the test set
    X_test[feature] = X_test[feature].map(
        lambda s: label_encoder.transform([s])[0] if s in label_encoder.classes_ else -1
    )
for feature in target_columns:
    # Fit and transform the training set
    y_train[feature] = label_encoder.fit_transform(y_train[feature])

    # Handle unknown labels in the test set
    y_test[feature] = y_test[feature].map(
        lambda s: label_encoder.transform([s])[0] if s in label_encoder.classes_ else -1
    )

# Display the preprocessed data
# Display the preprocessed data
print("X_train:", X_train.head())
print("y_train:", y_train.head())
print("X_test:", X_test.head())
print("y_test:", y_test.head())


X_train:           Age  Gender  Smoking History  Family History  Tumor Size  \
171 -0.425161       0                1               1   -1.565853   
71  -0.808719       0                0               0   -0.934142   
159  0.853366       0                0               1    0.907459   
105 -1.320130       0                1               0   -0.757476   
196 -0.744793       0                1               0    0.902106   

     Treatment Type  Blood Markers  IHC Results  Clinical Trial Participation  \
171               0            102           38                             0   
71                0             60           98                             0   
159               1             77           52                             1   
105               2             65           49                             0   
196               1             36           19                             0   

     Comorbidities  ECOG Performance Status  Symptom 1  Symptom 2  Symptom 3  \
171

In [26]:
import ssl
print(ssl.OPENSSL_VERSION)


OpenSSL 1.1.1w  11 Sep 2023


In [27]:
pip install --upgrade tensorflow


Defaulting to user installation because normal site-packages is not writeable
  Using cached urllib3-2.2.0-py3-none-any.whl (120 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.10
    Uninstalling urllib3-1.25.10:
      Successfully uninstalled urllib3-1.25.10
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.41 requires requests_mock, which is not installed.
botocore 1.27.59 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.0 which is incompatible.
conda-repo-cli 1.0.41 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.41 requires nbformat==5.4.0, but you have nbformat 5.7.0 which is incompatible.
conda-repo-cli 1.0.41 requires requests==2.28.1, but you have requests 2.31.0 which is incompatible.


In [28]:
!pip install keras


Defaulting to user installation because normal site-packages is not writeable


In [29]:
print(X_train.dtypes)
print("y_train shape", y_train.dtypes)


Age                             float64
Gender                            int32
Smoking History                   int32
Family History                    int32
Tumor Size                      float64
Treatment Type                    int32
Blood Markers                     int32
IHC Results                       int32
Clinical Trial Participation      int32
Comorbidities                     int32
ECOG Performance Status         float64
Symptom 1                         int32
Symptom 2                         int32
Symptom 3                         int32
Patient-reported Outcomes         int32
dtype: object
y_train shape Geneset              int32
Genetic Mutations    int32
Cancer Stage         int32
Histology Type       int32
Metastasis           int32
dtype: object


In [30]:
# Check the shape of X_train
print("Shape of X_train:", X_train.shape)

# Check the shape of y_train
print("Shape of y_train:", y_train.shape)
y_train = np.array(y_train)
print("numpy array of y_train :",y_train)
# Print unique values in y_train
print("Unique values in y_train:", np.unique(y_train))



Shape of X_train: (126, 15)
Shape of y_train: (126, 5)
numpy array of y_train : [[ 38  38   0   2   3]
 [ 98  98   3   0   1]
 [ 52  55   1   1   0]
 [ 49  49   1   1   3]
 [ 19  19   3   2   0]
 [ 39  39   1   2   3]
 [ 50  54   0   1   3]
 [ 34  34   1   2   3]
 [ 66  66   1   2   3]
 [ 37  37   2   2   1]
 [ 11  11   1   1   0]
 [113 113   3   1   3]
 [ 83  83   0   2   0]
 [119 119   2   1   0]
 [  0   0   3   0   2]
 [120 120   1   1   1]
 [ 63  63   0   1   3]
 [ 55  52   0   0   2]
 [100 100   3   0   1]
 [ 13  13   3   0   3]
 [112 112   0   1   2]
 [ 91  91   3   1   0]
 [121 121   2   1   3]
 [116 116   1   1   3]
 [ 23  23   1   2   0]
 [ 31  31   3   0   3]
 [105 105   3   1   3]
 [ 17  17   1   1   3]
 [106 106   2   2   2]
 [ 12  12   1   0   3]
 [ 99  99   3   0   3]
 [ 89  89   3   1   0]
 [104 104   3   1   0]
 [ 29  29   2   2   0]
 [ 81  81   3   1   0]
 [ 51  50   1   0   0]
 [ 47  47   1   1   2]
 [  4   4   2   1   0]
 [ 60  60   2   0   2]
 [ 16  16   2   2   3]


In [31]:
num_classes = len(np.unique(y_train))
num_classes

124

In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
import numpy as np

# Assuming y_train is numeric
# Convert y_train to NumPy array
y_train = np.array(y_train)

# Modify the last layer of your model for regression
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='linear'))  # Use 'linear' activation for regression

# Compile the model with 'mean_squared_error' loss for regression
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

# Display the model summary
model.summary()

# Train the DNN model using the training dataset
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


AttributeError: module 'lib' has no attribute 'X509_V_FLAG_NOTIFY_POLICY'

In [ ]:
# Tokenize and normalize symptoms
from sklearn.feature_extraction.text import TfidfVectorizer

def normalize_symptoms(text):
    tokens = text.lower().split()
    filtered_tokens = [token for token in tokens if token not in stops]
    normalized_tokens = [stemmer.stem(token) for token in filtered_tokens]
    return " ".join(normalized_tokens)

cancer_data['normalized_symptoms'] = cancer_data['symptoms'].apply(normalize_symptoms)
print(cancer_data['normalized_symptoms'][0])
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(cancer_data['normalized_symptoms'])
print(tfidf_matrix)
tfidf_matrix.shape


In [ ]:
# Gather user inputs
tfidf_features = vectorizer.transform(medical_transcript)
tfidf_features.shape

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Load pre-trained BioBERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("monologg/biobert_v1.1_pubmed")
model_bert = AutoModel.from_pretrained("monologg/biobert_v1.1_pubmed")

# Tokenize user input 
tokenized_user_input = tokenizer(user_inputs, is_split_into_words=True, padding=True, truncation=True, return_tensors="pt")

print(tokenized_user_input)
# Get word embeddings
with torch.no_grad():
    outputs = model_bert(**tokenized_user_input)
    word_embeddings = outputs.last_hidden_state.numpy()
word_embeddings.shape,word_embeddings

In [ ]:
# Load a model with NER capabilities (e.g., en_core_web_sm for general English)
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")
for user_input in user_inputs:
    doc = nlp(user_input)
    ## Render output by using displacy module
    displacy.render(doc, style="dep")
    # Extract entities and their labels
    entities = [(entity.text, entity.label_) for entity in doc.ents]
print(entities)
entity_types = [entity[1] for entity in entities]
encoder = OneHotEncoder(handle_unknown='ignore')
entities_features = encoder.fit_transform(np.array(entity_types).reshape(-1, 1)).toarray()
print(entities_features.shape)
print(entities_features)

In [ ]:
import spacy
from spacy import displacy
from sklearn.preprocessing import OneHotEncoder
import numpy as np

# Load a model with NER capabilities (e.g., en_core_web_sm for general English)
nlp = spacy.load("en_core_web_sm")

# List of user inputs (assuming you have it defined somewhere)
user_inputs = ["Your first user input", "Your second user input"]

# Extract entities and their labels for each user input
all_entities = []
for user_input in medical_transcript:
    doc = nlp(user_input)
    # Render output using displacy module (dependency visualization)
    displacy.render(doc, style="dep")
    
    # Extract entities and their labels
    entities = [(entity.text, entity.label_) for entity in doc.ents]
    all_entities.extend(entities)

# Print all entities
print("All entities:", all_entities)

# Extract entity types
entity_types = [entity[1] for entity in all_entities]

# Reshape entity types before using OneHotEncoder
entity_types_reshaped = np.array(entity_types).reshape(-1, 1)

# Initialize OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

# Fit and transform entity types
entities_features = encoder.fit_transform(entity_types_reshaped).toarray()

# Print the shape and features
print("Shape of entities features:", entities_features.shape)
print("Entities features:")
print(entities_features)


In [ ]:
import collections
from nltk import flatten
tokens_df= cancer_data['Geneset'] #drop null values
# Get top 30 tokens based upon frequency in whole corpus
word_freq_top30 = pd.DataFrame(collections.Counter(flatten(tokens_df.to_list())).most_common(),columns=['words',"frequency"])

In [ ]:
word_freq_top30

In [ ]:
##Plot bar graph of words wrt frequency 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(20,11)})
ax = sns.barplot(x=word_freq_top30['words'][1:], y=word_freq_top30['frequency'])
ax.set_xticklabels(ax.get_xticklabels(),rotation=75)
plt.xticks(rotation=45)

In [ ]:
# Combine the outputs from DNN and BioBERT
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Reshape

# Reshape model.output to have the same dimensions as word_embeddings
reshaped_dnn_output = Reshape((1, 1))(model.output)
print(reshaped_dnn_output.shape)
print(word_embeddings.shape)

In [ ]:
# Combine the outputs from DNN and BioBERT
from tensorflow.keras.layers import Concatenate, RepeatVector, Input, Reshape

input_shape = (10, 769)  # Adjust the size based on your actual input shapes

# Create an Input layer with the specified input shape
input_layer = Input(shape=input_shape)

# Repeat the vector to match the word_embeddings shape
reshaped_dnn_output = RepeatVector(512)(model.output)
print(reshaped_dnn_output.shape)  # Output: (None, 512, 1)

# Reshape to desired output
reshaped_dnn_output = Reshape((512, 1))(reshaped_dnn_output)
print(reshaped_dnn_output.shape)  # Output: (None, 512, 1)

# Now both reshaped_dnn_output and word_embeddings have compatible shapes
combined_features = Concatenate(axis=-1)([reshaped_dnn_output, word_embeddings])
print(combined_features.shape)

# Add a dense layer for further processing or prediction
combined_model = Sequential()
combined_model.add(Dense(64, activation='relu', input_shape=(512, 769)))  # Adjust input shape based on the combined_features.shape
combined_model.add(Dense(1, activation='linear'))  # Adjust activation based on the task (regression)

# Compile the combined model
combined_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

# Display the combined model summary
combined_model.summary()


In [ ]:
combined_features.shape

In [ ]:
combined_features

In [ ]:
print(type(combined_features))

In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
cancer_data.columns

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Concatenate, Input, Reshape

# Create an Input layer with the specified input shape
input_layer = Input(shape=(512, 769), name='input_layer')

# Design the final model architecture
dense_layer_1 = Dense(64, activation='relu')(input_layer)
dense_layer_2 = Dense(32, activation='relu')(dense_layer_1)

# Output layers for each target variable
geneset_output = Dense(1, activation='linear', name='geneset_output')(dense_layer_2)
mutation_output = Dense(1, activation='linear', name='mutation_output')(dense_layer_2)
histology_output = Dense(1, activation='linear', name='histology_output')(dense_layer_2)
cancer_stage_output = Dense(1, activation='linear', name='cancer_stage_output')(dense_layer_2)
metastasis_output = Dense(1, activation='linear', name='metastasis_output')(dense_layer_2)

# Create the final model
final_model = Model(inputs=input_layer, outputs=[geneset_output, mutation_output, histology_output,
                                                  cancer_stage_output, metastasis_output])

# Compile the final model
final_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

# Display the final model summary
final_model.summary()

# Train the final model using your data
final_model.fit(combined_features.numpy(),
                {'geneset_output': cancer_data['Geneset'], 
                 'mutation_output': cancer_data['Genetic Mutations'], 
                 'histology_output': cancer_data['Histology Type'],
                 'cancer_stage_output': cancer_data['Cancer Stage'],
                 'metastasis_output': cancer_data['Metastasis']},
                epochs=10, batch_size=32, validation_split=0.2)


In [ ]:
# Assuming you have a separate testing dataset
from sklearn.preprocessing import StandardScaler

def preprocess_data(data):
    # Assuming 'data' is a DataFrame
    numeric_features = ['Age', 'Tumor Size', 'ECOG Performance Status']
    
    # Standardize numerical features using StandardScaler
    scaler = StandardScaler()
    data[numeric_features] = scaler.transform(data[numeric_features])
    
    # Assuming 'text_columns' are the categorical features
    text_columns = ['Gender', 'Smoking History', 'Symptom 1', 'Symptom 2', 'Symptom 3',
                    'Treatment Type', 'Comorbidities', 'Clinical Trial Participation', 
                    'IHC Results', 'Blood Markers', 'Family History', 'Patient-reported Outcomes']
    
    # Encode categorical features using LabelEncoder
    label_encoder = LabelEncoder()
    for feature in text_columns:
        data[feature] = label_encoder.transform(data[feature])
    
    return data
x_data=preprocess_data(medical_transcript)
print(x_data)
X_test_preprocessed = preprocess_data(X_test)  # Apply the same preprocessing as during training

# Evaluate the model on the testing dataset
evaluation_results = final_model.evaluate(X_test_preprocessed,
                                          {'geneset_output': y_test['Geneset'], 
                                           'mutation_output': y_test['Genetic Mutations'], 
                                           'histology_output': y_test['Histology Type'],
                                           'cancer_stage_output': y_test['Cancer Stage'],
                                           'metastasis_output': y_test['Metastasis']},
                                          batch_size=32)

# Print the evaluation results
print("Evaluation Results:")
print("Loss:", evaluation_results[0])
print("Mean Squared Error (MSE):", evaluation_results[1])

# Assuming 'user_input' is the new data for prediction
user_input_preprocessed = preprocess_data(user_input)  # Apply the same preprocessing as during training

# Make predictions on the new data
predictions = final_model.predict(user_input_preprocessed)

# Display the predictions
print("Predictions:")
print("Geneset Output:", predictions[0][0])
print("Mutation Output:", predictions[1][0])
print("Histology Output:", predictions[2][0])
print("Cancer Stage Output:", predictions[3][0])
print("Metastasis Output:", predictions[4][0])


In [ ]:
Feature extraction typically occurs during the training phase of a machine learning model. In the context of deep learning models like DNNs, feature extraction is an inherent part of the training process. During training, the model learns to extract relevant features from the input data to make predictions.

Here's a more detailed breakdown of the process:

1)Data Splitting:

Split your dataset into training and testing sets. This is done at the beginning to assess the model's performance on unseen data.
Common splits include 80% of the data for training and 20% for testing.
2.)Data Preprocessing:

Preprocess both the gene and transcript data and the medical transcript data. This involves normalizing, scaling, or encoding features as necessary.
3.)DNN Model Training:

Design and define the architecture of your DNN model, considering the specific characteristics of your gene and transcript data.
Compile the model with an appropriate optimizer, loss function, and evaluation metric.
Train the DNN model using the training dataset.
The DNN learns to automatically extract relevant features during the training process.
4.)BioBERT Fine-tuning:

Fine-tune BioBERT on your medical transcript data during the training phase.
Extract features from the BioBERT model, either during fine-tuning or later.
5.)Integration of DNN and BioBERT:

Combine the outputs from the DNN model (gene and transcript data) with the features extracted from BioBERT (medical transcript data).
This integration step can happen after training both the DNN and BioBERT.
6.)Final Model Training:

Design a final model that takes the integrated data as input and predicts geneset name, genetic mutation, and histology type related to lung cancer.
Train the final model using the integrated data.
7.)Model Evaluation:

Evaluate the performance of the final model on the testing dataset.
Assess how well the model generalizes to new, unseen data.
Feature Extraction during Testing/User Input:

When using the model for predictions on new, unseen data (testing or user input), feature extraction is still part of the inference process.
The model automatically extracts relevant features from the input data to make predictions.
In summary, feature extraction is an intrinsic part of the training and prediction process in machine learning models. During training, the model learns to extract features, and during testing or inference, it applies this knowledge to new data for making predictions.